# Setup

In [1]:
## !pip install numpy scipy

In [2]:
import collections
import functools as ft
import itertools as it
import multiprocessing as mp
import numpy as np
import pandas as pd
import scipy.optimize as so
import scipy.stats as st

In [3]:
%matplotlib inline
%config InlineBackend.figure_formats = ['svg']
import matplotlib.pyplot as plt

In [4]:
F = st.uniform(0, 1)

# Implementation

In [5]:
def revenue(values, virtual_values):
    return max(np.array(values).max(), np.array(virtual_values).max())

def trial(y, cost, seed, num_players, F):
    np.random.seed(seed)
    values = np.random.uniform(0, 1, size=num_players)
    virtual_values = values - (1 - F.cdf(values) / F.pdf(values))
    ret = dict()
    ## Important: this points to the first *unselected* agent
    for n in range(num_players - 1):
        ret[n] = revenue(max(values[n], y), virtual_values[n + 1:]) - cost - revenue([y], virtual_values[n:])
    ret[num_players - 1] = revenue([max(values[-1], y)], [-np.inf]) - cost - revenue([y], virtual_values[-1:])
    return y, cost, seed, values, ret

In [6]:
def run_auction(ys, costs, num_players, F, num_trials=10_000, seed=0):
    values = F.rvs(size=(num_trials, num_players))
    virtual_values = values - (1 - F.cdf(values) / F.pdf(values))
    ys = ys.reshape(1, -1)
    costs = costs.reshape(1, 1, -1)
    gittins_index = dict()
    reservation_values = np.zeros((num_trials, costs.shape[-1]))
    total_cost = np.zeros_like(reservation_values)
    halt = np.zeros((num_trials, costs.shape[-1]), dtype=bool)
    for j in range(num_players):
        # trials x ys x costs
        expected_improvement = (np.fmax(ys, np.fmax(values[:,j], virtual_values[:,j + 1:].max(axis=1, initial=0)).reshape(-1, 1))
                                - np.fmax(ys, virtual_values[:,j:].max(axis=1, initial=0).reshape(-1, 1))).reshape(num_trials, ys.shape[1], 1) - costs
        signs = np.sign(expected_improvement.mean(axis=0))
        hits = np.where(np.diff(signs, axis=0))
        gittins_index[j] = pd.Series(dict(zip(costs.ravel()[hits[1]], ys.ravel()[hits[0]])), index=costs.ravel()).fillna(np.inf * pd.Series(signs[0], index=costs.ravel()))
        halt |= reservation_values > gittins_index[j].values.reshape(1, -1)
        total_cost += np.where(halt, 0, costs.ravel())
        reservation_values = np.where(halt, 
                                      np.fmax(reservation_values, virtual_values.max(axis=1).reshape(-1, 1)), 
                                      np.fmax(reservation_values, values[:,j].reshape(-1, 1)))
    revenues = reservation_values - total_cost
    gittins_index = pd.DataFrame(gittins_index)
    gittins_index.index.name = 'cost'
    gittins_index.columns.name = 'num_inspected'
    revenue = pd.Series(revenues.mean(axis=0), index=costs.ravel())
    return gittins_index, revenue

# Results

In [ ]:
batch_size = 1_000
revenues = dict()
gittins_index = dict()
for batch in range(30):
    gittins_index[batch], revenues[batch] = run_auction(np.linspace(0, 1, 1000), np.logspace(-3, -1, 100), 5, F, num_trials=batch_size, seed=batch)

In [ ]:
revenue = pd.DataFrame(revenues).mean(axis=1)

In [ ]:
plt.clf()
plt.gcf().set_size_inches(6, 3)
plt.plot(revenue.index, revenue, lw=1, c='k')
plt.xlabel('Cost')
plt.ylabel('Expected revenue')
plt.xscale('log')
plt.tight_layout()

# Slow, readable code

In [ ]:
with mp.Pool() as pool:
    temp = pool.starmap(ft.partial(trial, num_players=5, F=F), 
                        it.product(np.linspace(0, 1, 100), [1e-2], range(50_000)))
result = collections.defaultdict(lambda: collections.defaultdict(dict))
for y, c, t, v, f in temp:
    for k in f:
        result[(k, c)][t][y] = f[k]

In [ ]:
index = collections.defaultdict(dict)
index_sd = collections.defaultdict(dict)
for k, c in result:
    mean = pd.DataFrame(result[(k, c)]).mean(axis=1)
    std = pd.DataFrame(result[(k, c)]).std(axis=1)
    hit = np.where(np.diff(np.sign(mean)))[0]
    if hit.any():
        index[c][k] = mean.index[hit].item()
        index_sd[c][k] = std.iloc[hit]
    elif mean[0] < 0:
        index[c][k] = -np.inf
        index_sd[c][k] = 0
    else:
        index[c][k] = np.inf
        index_sd[c][k] = 0

In [ ]:
gittins_index_table = pd.DataFrame(index)
gittins_index_table.index.name = 'num_inspected'
gittins_index_table.columns.name = 'cost'
gittins_index_table